In [25]:
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
)

import torch, copy, os
from module.score import evaluation_f1
from module.load_json import *
from module.maps import *
from module.args import print_torch_info
from module.inference import *

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
print_torch_info()

torch.__version__: 1.7.1
torch.cuda.is_available(): True
NGPU: 4


In [26]:
ACD_1_CHECKPOINT = 'training_results/uncleaned_v22_run_1/acd_klue_roberta_base_mlm_fine_tuned_uncleaned_v22_run_1/checkpoint-37500'
ACD_2_CHECKPOINT = 'training_results/uncleaned_v22_run_1/acd_monologg_koelectra_base_v3_discriminator_uncleaned_v22_run_1/checkpoint-37500'
ACD_3_CHECKPOINT = 'training_results/uncleaned_v22_run_1/acd_snunlp_kr_electra_discriminator_uncleaned_v22_run_1/checkpoint-37500'

ASC_1_CHECKPOINT = 'training_results/uncleaned_v22_run_1/asc_klue_roberta_base_mlm_fine_tuned_uncleaned_v22_run_1/checkpoint-1608'
ASC_2_CHECKPOINT = 'training_results/uncleaned_v22_run_1/asc_monologg_koelectra_base_v3_discriminator_uncleaned_v22_run_1/checkpoint-1608'
ASC_3_CHECKPOINT = 'training_results/uncleaned_v22_run_1/asc_snunlp_kr_electra_discriminator_uncleaned_v22_run_1/checkpoint-1608'

In [27]:
acd_model_1 = AutoModelForSequenceClassification.from_pretrained(ACD_1_CHECKPOINT)
acd_model_2 = AutoModelForSequenceClassification.from_pretrained(ACD_2_CHECKPOINT)
acd_model_3 = AutoModelForSequenceClassification.from_pretrained(ACD_3_CHECKPOINT)
acd_tokenizer_1 = AutoTokenizer.from_pretrained(ACD_1_CHECKPOINT)
acd_tokenizer_2 = AutoTokenizer.from_pretrained(ACD_2_CHECKPOINT)
acd_tokenizer_3 = AutoTokenizer.from_pretrained(ACD_3_CHECKPOINT)

asc_model_1 = AutoModelForSequenceClassification.from_pretrained(ASC_1_CHECKPOINT)
asc_model_2 = AutoModelForSequenceClassification.from_pretrained(ASC_2_CHECKPOINT)
asc_model_3 = AutoModelForSequenceClassification.from_pretrained(ASC_3_CHECKPOINT)
asc_tokenizer_1 = AutoTokenizer.from_pretrained(ASC_1_CHECKPOINT)
asc_tokenizer_2 = AutoTokenizer.from_pretrained(ASC_2_CHECKPOINT)
asc_tokenizer_3 = AutoTokenizer.from_pretrained(ASC_3_CHECKPOINT)

# asc_model = AutoModelForSequenceClassification.from_pretrained(ASC_CHECKPOINT)
# asc_tokenizer = AutoTokenizer.from_pretrained(ASC_CHECKPOINT)

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [29]:
form = '야 이놈아 이 것 좀 보려무나!'
acd_pair = '이놈#저놈'

In [38]:
# # v1

# def acd_ensemble(form, acd_pair):
#     acd_model_1.to(device)
#     acd_model_2.to(device)
#     acd_model_3.to(device)
#     acd_model_1.eval()
#     acd_model_2.eval()
#     acd_model_3.eval()

#     acd_encoded_1 = acd_tokenizer_1(form, acd_pair, truncation=True, return_tensors="pt")
#     acd_encoded_1 = {k:v.to(device) for k,v in acd_encoded_1.items()}

#     acd_encoded_2 = acd_tokenizer_2(form, acd_pair, truncation=True, return_tensors="pt")
#     acd_encoded_2 = {k:v.to(device) for k,v in acd_encoded_2.items()}

#     acd_encoded_3 = acd_tokenizer_3(form, acd_pair, truncation=True, return_tensors="pt")
#     acd_encoded_3 = {k:v.to(device) for k,v in acd_encoded_3.items()}

#     with torch.no_grad():
#         acd_outputs_1 = acd_model_1(**acd_encoded_1)['logits']
#         acd_outputs_2 = acd_model_2(**acd_encoded_2)['logits']
#         acd_outputs_3 = acd_model_3(**acd_encoded_3)['logits']

#     acd_outputs = acd_outputs_1 + acd_outputs_2 + acd_outputs_3
#     acd_predictions = acd_outputs.argmax(-1)[0]
    
#     return acd_predictions

In [39]:
# v2

def acd_ensemble(form, acd_pair):
    acd_model_1.to(device)
    acd_model_2.to(device)
    acd_model_3.to(device)
    acd_model_1.eval()
    acd_model_2.eval()
    acd_model_3.eval()

    acd_encoded_1 = acd_tokenizer_1(form, acd_pair, truncation=True, return_tensors="pt")
    acd_encoded_1 = {k:v.to(device) for k,v in acd_encoded_1.items()}

    acd_encoded_2 = acd_tokenizer_2(form, acd_pair, truncation=True, return_tensors="pt")
    acd_encoded_2 = {k:v.to(device) for k,v in acd_encoded_2.items()}

    acd_encoded_3 = acd_tokenizer_3(form, acd_pair, truncation=True, return_tensors="pt")
    acd_encoded_3 = {k:v.to(device) for k,v in acd_encoded_3.items()}

    with torch.no_grad():
        acd_outputs_1 = acd_model_1(**acd_encoded_1)['logits']
        acd_outputs_2 = acd_model_2(**acd_encoded_2)['logits']
        acd_outputs_3 = acd_model_3(**acd_encoded_3)['logits']

    acd_outputs = acd_outputs_1 + acd_outputs_2 + acd_outputs_3
    
    acd_predictions_1 = acd_outputs_1.argmax(-1).squeeze()
    acd_predictions_2 = acd_outputs_2.argmax(-1).squeeze()
    acd_predictions_3 = acd_outputs_3.argmax(-1).squeeze()
    print([acd_predictions_1, acd_predictions_2, acd_predictions_3])
    if 0 in [acd_predictions_1, acd_predictions_2, acd_predictions_3]:
        return 0
    
    return 1

In [40]:
form = ''
acd_pair = '헙헙#요요홉'

In [41]:
acd_ensemble(form, acd_pair)

[tensor(0, device='cuda:0'), tensor(1, device='cuda:0'), tensor(1, device='cuda:0')]


0

In [5]:
def asc_ensemble(form, asc_pair):
    asc_model_1.to(device)
    asc_model_2.to(device)
    asc_model_3.to(device)
    asc_model_1.eval()
    asc_model_2.eval()
    asc_model_3.eval()

    asc_encoded_1 = asc_tokenizer_1(form, asc_pair, truncation=True, return_tensors="pt")
    asc_encoded_1 = {k:v.to(device) for k,v in asc_encoded_1.items()}

    asc_encoded_2 = asc_tokenizer_2(form, asc_pair, truncation=True, return_tensors="pt")
    asc_encoded_2 = {k:v.to(device) for k,v in asc_encoded_2.items()}

    asc_encoded_3 = asc_tokenizer_3(form, asc_pair, truncation=True, return_tensors="pt")
    asc_encoded_3 = {k:v.to(device) for k,v in asc_encoded_3.items()}

    with torch.no_grad():
        asc_outputs_1 = asc_model_1(**asc_encoded_1)['logits']
        asc_outputs_2 = asc_model_2(**asc_encoded_2)['logits']
        asc_outputs_3 = asc_model_3(**asc_encoded_3)['logits']

    asc_outputs = asc_outputs_1 + asc_outputs_2 + asc_outputs_3
    
    return asc_outputs

In [ ]:
def inference_m_ensemble(acd_ensemble, asc_ensemble, data, entity_property_pair):
    print(entity_property_pair)
    for sentence in tqdm(data):
        form = sentence['sentence_form']
        sentence['annotation'] = []
        if type(form) != str:
            print("form type is wrong: ", form)
            continue
        for pair in entity_property_pair:
            acd_pair = pair

            acd_predictions = acd_ensemble(form, acd_pair)
            acd_result = tf_id_to_name[acd_predictions]

            if acd_result == 'True':
                asc_pair = pair

                asc_outputs = asc_ensemble(form, asc_pair)
                asc_predictions = asc_outputs.argmax(-1)
                asc_result = polarity_id_to_name[asc_predictions[0]]

                if pair == '패키지/구성품#가격':
                    print(f'{pair} found.')
                    pair = '패키지/ 구성품#가격'
                    print(f'corrected as {pair}')

                sentence['annotation'].append([pair, asc_result])

    return data